# 1. 패키지 로드

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from PIL import Image
from tqdm import tqdm
import time
import gc
import random
from box import Box
import cv2
import cvlib as cv
import timm

import warnings
warnings.filterwarnings('ignore')

# 2. 학습 관련 함수

## 2-1. seed 고정

In [ ]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

## 2-2. 데이터 전처리

In [ ]:
# ages 생성
def get_ages(x):
    if x < 30: return 0
    elif x < 60: return 1
    else: return 2

# genders 생성
def get_genders(x):
    if x == 'male': return 0
    else: return 1

# masks 생성
def get_masks(x):
    if x == 'normal': return 2
    elif x == 'incorrect_mask': return 1
    else: return 0

# # age_cats 생성
# def get_age_cats(x):
#     if x < 20: return 0
#     elif x < 30: return 1
#     elif x < 40: return 2
#     elif x < 50: return 3
#     elif x < 60: return 4
#     else: return 5

def get_age_cats(x):
    if x < 25: return 0
    elif x < 30: return 1
    elif x < 45: return 2
    elif x < 52: return 3
    elif x < 57: return 4
    elif x < 60: return 5
    else: return 6

# labels 생성
def get_labels(masks, genders, ages):
    return masks * 6 + genders * 3 + ages

# label_cats 생성
def get_label_cats(masks, genders, ages):
    return masks * 12 + genders * 6 + ages

# 마스크 이상치 변경
def swap_mask(swap_li : list, df : pd.DataFrame) -> pd.DataFrame:
    swap_df = df.copy()
    for swap_id in swap_li:
        _swap_df = swap_df[swap_df['id'] == swap_id]
        
        normal_swap_df = _swap_df[_swap_df['mask'] == 'normal']
        incorrect_mask_swap_df = _swap_df[_swap_df['mask'] == 'incorrect_mask']
        
        normal_path = normal_swap_df['path'].values[0]
        incorrect_mask_path = incorrect_mask_swap_df['path'].values[0]
        
        swap_df.loc[normal_swap_df.index, 'path'] = incorrect_mask_path
        swap_df.loc[incorrect_mask_swap_df.index, 'path'] = normal_path
    
    return swap_df

# train_df + mask 결측치 처리
def make_train_df(df : pd.DataFrame, swap_mask_li : list, cfg) -> pd.DataFrame:
    train_df = []
    
    for line in df.iloc:
        for file in list(os.listdir(os.path.join(cfg.train_image_dir, line['path']))):
            if file[0] == '.':
                continue
            
            mask = file.split('.')[0]
            gender = line['gender']
            age = line['age']
            
            masks = get_masks(mask)
            genders = get_genders(gender)
            ages = get_ages(age)
            age_cats = get_age_cats(age)
            
            data = {
                'id' : line['id'],
                'mask' : mask,
                'gender' : gender,
                'age' : age,
                'masks' : masks,
                'genders' : genders,
                'ages' : ages,
                'age_cats' : age_cats,
                'labels': get_labels(masks = masks, genders = genders, ages = ages),
                'label_cats': get_label_cats(masks = masks, genders = genders, ages = age_cats),
                'path': os.path.join(cfg.train_image_dir, line['path'], file),
            }
            
            train_df.append(data)
            
    train_df = pd.DataFrame(train_df)
    
    train_df['idx'] = train_df.index
    
    train_df = swap_mask(swap_li = swap_mask_li, df = train_df)
    
    return train_df

# 성별 이상치 처리
def swap_gender(swap_li : list, df : pd.DataFrame) -> pd.DataFrame:
    swap_df = df.copy()
    for swap in swap_li:
        swap_id, swap_gender = swap
        swap_df.loc[swap_df[swap_df['id'] == swap_id].index, 'gender'] = swap_gender
    return swap_df

# 사람 나누기 데이터 + 성별 결측치 처리
def preprocessing_df(df : pd.DataFrame, swap_gender_li : list) -> pd.DataFrame:
    
    preprocessing_df = df.copy()
    preprocessing_df = swap_gender(swap_li = swap_gender_li, df = preprocessing_df)
    
    preprocessing_df['ages'] = preprocessing_df['age'].apply(lambda x : get_ages(x))
    preprocessing_df['genders'] = preprocessing_df['gender'].apply(lambda x : get_genders(x))
    
    preprocessing_df['cv_taget_col'] = 'age' + '_' + preprocessing_df['age'].astype(str) + '_' + 'genders' + '_' + preprocessing_df['genders'].astype(str)
    
    return preprocessing_df

## 2-3 이상치 시각화

In [ ]:
# 이상치 이미지 시각화
def show_img(img_id_li, df, cfg):
    for img_id in img_id_li:
        get_df = df[df['id'] == img_id]
        
        img_age = get_df['age'].tolist()[0]
        img_gender = get_df['gender'].tolist()[0]
        
        img_path = get_df['path'].tolist()[0]
        img_path = os.path.join(cfg.train_image_dir, img_path)
        img_name_li = sorted(list(os.listdir(img_path)))
        
        fig, ax = plt.subplots(1, 7, figsize = (30, 15))
        ax = ax.flatten()
        
        idx = 0
        for _img_name in img_name_li:
            if _img_name[0] == '.': continue
            
            if _img_name.split('.')[0] == 'normal': imag_name = 'normal'
            elif _img_name.split('.')[0] == 'incorrect_mask': imag_name = 'incorrect_mask'
            else: imag_name = 'mask'
            
            get_img_path = os.path.join(img_path, _img_name)
            
            img = Image.open(get_img_path)
            img = np.array(img)
            ax[idx].imshow(img)
            ax[idx].set_title(f'{img_id} / {img_age} / {img_gender} / {imag_name}')
            ax[idx].set_xticks([])
            ax[idx].set_yticks([])
            idx += 1
            
        plt.show()

# image path로 이미지 시각화
def path_li_show_img(path_li):
    fig, ax = plt.subplots(1, 7, figsize = (30, 15))
    ax = ax.flatten()
    idx = 0
    for path in path_li:
        image_name = path.split('/')[-1]
        img = Image.open(path)
        img = np.array(img)
        ax[idx].imshow(img)
        ax[idx].set_title(f'{image_name}')
        ax[idx].set_xticks([])
        ax[idx].set_yticks([])
        idx += 1
    plt.show()

## 2-4. 데이터 스플릿

In [ ]:
# val_idx 생성
def get_val_idx(df : pd.DataFrame, target_col : str):
    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 22)
    for trn_idx, val_idx in skf.split(df, df[target_col]):
        yield val_idx

class StratifiedSampler(torch.utils.data.Sampler):
    """Stratified batch sampling
    Provides equal representation of target classes in each batch
    """
    def __init__(self, y, batch_size, shuffle=True):
        if torch.is_tensor(y):
            y = y.cpu().numpy()
        assert len(y.shape) == 1, 'label array must be 1D'
        n_batches = int(len(y) / batch_size)
        self.skf = StratifiedKFold(n_splits = n_batches, shuffle = shuffle)
        self.X = torch.randn(len(y),1).numpy()
        self.y = y
        self.shuffle = shuffle

    def __iter__(self):
        if self.shuffle:
            self.skf.random_state = torch.randint(0,int(1e8),size=()).item()
        for train_idx, test_idx in self.skf.split(self.X, self.y):
            yield test_idx

    def __len__(self):
        return len(self.y)

## 2-5. 모델 관련

In [ ]:
class CreateModel(nn.Module):
    def __init__(self, cfg , pretrained : bool = True):
        super(CreateModel, self).__init__()
        self.model = timm.create_model(cfg.timm_model_name, pretrained = pretrained, num_classes = cfg.num_classes)

    def forward(self, img):
        out = self.model(img)
        return out

In [ ]:
def model_train(model, optimizer, criterion, data_loader):
    model.train()
    
    train_loss = 0
    real_pred_li = []
    label_pred_li = []
    
    for images, targets in data_loader:
        images, targets = images.to(device), targets.to(device)
        optimizer.zero_grad()
        
        # GA 추가시 아래 부분에 추가하기
        #############################
        
        benign_outputs = model(images)
        loss = criterion(benign_outputs, targets)
        loss.backward()
        
        optimizer.step()
        
        train_loss += loss.item()
        
        predicted = benign_outputs.argmax(dim=-1)
        
        label_pred_li.extend(predicted.detach().cpu().numpy())
        real_pred_li.extend(targets.cpu().numpy())
        
#     label_pred_li = [label_cats2labels[i] for i in label_pred_li]
#     real_pred_li = [label_cats2labels[i] for i in real_pred_li]
    
    train_loss /= len(data_loader)
    train_acc = get_acc_score(y_true = real_pred_li, y_pred = label_pred_li)
    train_fi_score = get_f1_score(y_true = real_pred_li, y_pred = label_pred_li)

    return train_loss, train_acc, train_fi_score

def model_eval(model, criterion, data_loader):
    model.eval()
    
    val_loss = 0
    real_pred_li = []
    label_pred_li = []
    
    with torch.no_grad():
        for images, targets in data_loader:
            images, targets = images.to(device), targets.to(device)

            benign_outputs = model(images)
            loss = criterion(benign_outputs, targets)

            val_loss += loss.item()

            predicted = benign_outputs.argmax(dim=-1)
        
            label_pred_li.extend(predicted.cpu().numpy())
            real_pred_li.extend(targets.cpu().numpy())
    
#     label_pred_li = [label_cats2labels[i] for i in label_pred_li]
#     real_pred_li = [label_cats2labels[i] for i in real_pred_li]
    
    val_loss /= len(data_loader)
    val_acc = get_acc_score(y_true = real_pred_li, y_pred = label_pred_li)
    val_fi_score = get_f1_score(y_true = real_pred_li, y_pred = label_pred_li)
   
    return val_loss, val_acc, val_fi_score

def get_val_pred_li(model, data_loader):
    model.eval()
    real_pred_li = []
    label_pred_li = []
    ensemble_pred_li = []
    
    with torch.no_grad():
        for images, targets in data_loader:
            images = images.to(device)
            output = model(images)
            
            label = output.argmax(dim=-1)
            label_pred_li.extend(label.cpu().numpy())
            
            ensemble_label = output.softmax(1)
            ensemble_pred_li.append(ensemble_label.cpu().numpy())
            
            real_pred_li.extend(targets.cpu().numpy())
            
#     label_pred_li = [label_cats2labels[i] for i in label_pred_li]
#     real_pred_li = [label_cats2labels[i] for i in real_pred_li]
    
    return label_pred_li, np.concatenate(ensemble_pred_li), real_pred_li

def get_submission_pred_li(model, data_loader):
    model.eval()
    label_pred_li = []
    ensemble_pred_li = []
    
    with torch.no_grad():
        for images in data_loader:
            images = images.to(device)
            output = model(images)
            
            label = output.argmax(dim=-1)
            label_pred_li.extend(label.cpu().numpy())
            
            ensemble_label = output.softmax(1)
            ensemble_pred_li.append(ensemble_label.cpu().numpy())
            
#     label_pred_li = [label_cats2labels[i] for i in label_pred_li]
    
    return label_pred_li, np.concatenate(ensemble_pred_li)

## 2-6. 평가

In [ ]:
def get_f1_score(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro')

def get_acc_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

## 2-7. 로스

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, weight=None, gamma=2., reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )


class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes=3, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))


# https://gist.github.com/SuperShinyEyes/dcc68a08ff8b615442e3bc6a9b55a354
class F1Loss(nn.Module):
    def __init__(self, classes=3, epsilon=1e-7):
        super().__init__()
        self.classes = classes
        self.epsilon = epsilon

    def forward(self, y_pred, y_true):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, self.classes).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)

        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2 * (precision * recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1 - self.epsilon)
        return 1 - f1.mean()

# 3. 학습 환경 설정

## 3-1. config

In [ ]:
v = 2
config = {
    'seed' : 22,
    'data_split_col' : 'age',
    'oof' : 5,
    'tagets_col' : 'labels',
    
    'train_data_name' : 'train.csv',
    'train_data_dir' : '/opt/ml/input/data/train',
    'train_image_dir' : '/opt/ml/input/data/train/images',
    
    'submission_data_name' : 'info.csv',
    'submission_data_dir' : '/opt/ml/input/data/eval',
    'submission_image_dir' : '/opt/ml/input/data/eval/images',
    
    'file_name' : f'Ensembel_v{v}.csv',
    
    'mainModel':{
        
        'tagets_col' : 'labels',
        'split_col' : 'labels',
        
        'timm_model_name' : 'regnetx_002',
        
        'model_dir' : '/opt/ml/model',
        'model_name' : f'main_model_v{v}',
        'num_workers' : 4,
        'epochs' : 20,
        'batch_size' : 64,
        'lr' : 9e-05,
        'num_classes' : 18,
        
        'loss' : 'cel',
        'smoothing' : 0.1,
        
        'image_size' : [512, 384],
        'image_normal_mean' : [0.5, 0.5, 0.5],
        'image_normal_std' : [0.2, 0.2, 0.2],
        
    },
    
    'maleAgeModel':{
        
        'tagets_col' : 'ages',
        'split_col' : 'ages',
        
        'timm_model_name' : 'regnetx_002',
        
        'model_dir' : '/opt/ml/model',
        'model_name' : f'male_age_model_v{v}',
        'num_workers' : 4,
        'epochs' : 20,
        'batch_size' : 64,
        'lr' : 9e-05,
        'num_classes' : 3,
        
        'loss' : 'cel',
        'smoothing' : 0.1,
        
        'image_size' : [512, 384],
        'image_normal_mean' : [0.5, 0.5, 0.5],
        'image_normal_std' : [0.2, 0.2, 0.2],
        
    },
    
    'femaleAgeModel':{
        
        'tagets_col' : 'ages',
        'split_col' : 'ages',
        
        'timm_model_name' : 'regnetx_002',
        
        'model_dir' : '/opt/ml/model',
        'model_name' : f'female_age_model_v{v}',
        'num_workers' : 4,
        'epochs' : 20,
        'batch_size' : 64,
        'lr' : 9e-05,
        'num_classes' : 3,
        
        'loss' : 'cel',
        'smoothing' : 0.1,
        
        'image_size' : [512, 384],
        'image_normal_mean' : [0.5, 0.5, 0.5],
        'image_normal_std' : [0.2, 0.2, 0.2],
        
    },
    
}

config = Box(config)

## 3-2 이미지

In [ ]:
# 변환할 transform
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize, Lambda, RandomHorizontalFlip, ToPILImage, CenterCrop, Grayscale

def image_face_crop(image):
    image = np.array(image)
    face, confidence = cv.detect_face(image)
    if not face : return image
    x, y, w, h = face[0]
    H, W, C = image.shape
    image = image[max(y - 100, 0) : min(h + 100, H), max(0 , x - 100) : min(w + 100, W)]
    return image

transform = {
        "mainModel": transforms.Compose(
            [
#                 Resize(config.maskModel.image_size, Image.BILINEAR),
                ToTensor(),
                Normalize(mean=config.mainModel.image_normal_mean, std=config.mainModel.image_normal_std),
                Grayscale(num_output_channels = 3)
            ]
        ),
        "maleAgeModel": transforms.Compose(
            [
#                 Resize(config.genderModel.image_size, Image.BILINEAR),
                ToTensor(),
                Normalize(mean=config.maleAgeModel.image_normal_mean, std=config.maleAgeModel.image_normal_std),
                Grayscale(num_output_channels = 3)
            ]
        ),
        "femaleAgeModel": transforms.Compose(
            [
#                 Resize(config.ageModel.image_size, Image.BILINEAR),
                ToTensor(),
                Normalize(mean=config.femaleAgeModel.image_normal_mean, std=config.femaleAgeModel.image_normal_std),
                Grayscale(num_output_channels = 3)
            ]
        ),
    
}

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df : pd.DataFrame, cfg, transform = None, mode : bool = True, mode_type = None, arg_transform = None):
        self.mode = mode
        self.df = df
        self.mode_type = mode_type
        if self.mode:
            if self.mode_type =='arg':
                self.img_paths = self.df['path'].tolist()
                self.targets = self.df[cfg.tagets_col].tolist()
                self.split_targets = self.df[cfg.split_col].tolist()
                self.arg_cols = self.df['arg_types'].tolist()
            else:
                self.img_paths = self.df['path'].tolist()
                self.targets = self.df[cfg.tagets_col].tolist()
                self.split_targets = self.df[cfg.split_col].tolist()
        else:
            self.img_paths = [os.path.join(cfg.submission_image_dir, img_id) for img_id in self.df.ImageID]
        self.transform = transform
        self.arg_transform = arg_transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])
        if self.mode_type =='arg':
            if self.arg_cols[index] == 0:
                image = self.transform(image)
            else:
                image = self.arg_transform(image)
        else:
            if self.transform:
                image = self.transform(image)
        
        # 이 부분에 해당 라벨에 따른 데이터 변환 여부 추가
        # val 데이터의 경우 데이터 변환이 일어나면 안되기 때문에
        # if self.데이터 변환해주는 transform:
        #     if self.targets[index].data == labels: <- 확률적으로
        #          image = self.데이터 변환해주는 transform(image)
        # 데이터 변환
        
        if self.mode:
            targets = torch.tensor(self.targets[index])
            return image, targets
        
        else: return image

    def __len__(self):
        return len(self.img_paths)

In [ ]:
def get_trn_val_dataset_dataloader(trn_df : pd.DataFrame, val_df : pd.DataFrame, cfg, model_name : str):
    
    trn_dataset = CustomDataset(df = trn_df,
                            cfg = cfg,
                            transform = transform[model_name],
                            mode = True,
                           )
    
    train_loader = DataLoader(trn_dataset,
                       batch_size = cfg.batch_size,
                       num_workers = cfg.num_workers,
                       shuffle = True,
                        )

    val_dataset = CustomDataset(df = val_df,
                            cfg = cfg,
                            transform = transform[model_name],
                            mode = True,)

    val_loader = DataLoader(val_dataset,
                       batch_size = cfg.batch_size,
                       num_workers = cfg.num_workers,
                       shuffle = False,)

    return trn_dataset, train_loader, val_dataset, val_loader

def get_criterion(config):
    # loss 설정
    if config.loss == 'cel':
        criterion = nn.CrossEntropyLoss()
    elif config.loss == 'labelsmoothing':
        criterion = LabelSmoothingLoss(classes=config.num_classes, smoothing = config.smoothing, dim=-1)
    elif config.loss == 'focal':
        criterion = FocalLoss(weight = config.weight, gamma=2.0, reduction='mean')
    elif config.loss == 'f1':
        criterion = F1Loss(classes=config.num_classes, epsilon=1e-7)
    else:
        print('not loss')
    return criterion

def train(model, optimizer, criterion, train_loader, val_loader, scheduler, config, version = ''):
    besf_f1 = 0
    for epoch in range(1, config.epochs + 1):
        epoch_start_time = time.time()

        train_loss, train_acc, train_fi_score = model_train(model = model, 
                                                        optimizer = optimizer, 
                                                        criterion = criterion, 
                                                        data_loader = train_loader)

        val_loss, val_acc, val_fi_score, = model_eval(model = model,
                                                  criterion = criterion,
                                                  data_loader = val_loader)

        now_lr = get_lr(optimizer = optimizer)

        epoch_end_time = time.time()

        print(f'''{fold_num}fold, epoch: {epoch}, lr: {now_lr}, train_loss: {train_loss:.4f}, train_acc: {train_acc:.4f}, train_f1: {train_fi_score:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}, val_fi: {val_fi_score:.4f}, 학습시간: {epoch_end_time - epoch_start_time} \n''')

        scheduler.step(val_loss)

        if besf_f1 < val_fi_score:
            besf_f1 = val_fi_score
            torch.save(model.state_dict(), os.path.join(config.model_dir, f'{version}_{fold_num}fold_{config.model_name}.pt'))
            print(val_fi_score, '모델 저장')
            
def get_eval_img_show(train_df, idx_li, pred_labels, config):
    image_show_df = train_df.set_index('idx').loc[idx_li, :].reset_index(drop = True)
    image_show_df['pred_labels'] = pred_labels
    false_image_show_df = image_show_df[image_show_df[config.tagets_col] != image_show_df['pred_labels']]
    labels_li = [i for i in range(config.num_classes)]
    
    for labels in labels_li:
        _false_image_show_df = false_image_show_df[false_image_show_df[config.tagets_col] == labels]
        path_labels_pred_labels_li = _false_image_show_df[['path', config.tagets_col, 'pred_labels']].values[:7]

        idx = 0
        fig, ax = plt.subplots(1, 7, figsize = (30, 15))
        ax = ax.flatten()
        for path_labels_pred_labels in path_labels_pred_labels_li:
            img = Image.open(path_labels_pred_labels[0])
            img = np.array(img)
            ax[idx].imshow(img)
            ax[idx].set_title(f'true {path_labels_pred_labels[1]} / pred {path_labels_pred_labels[2]}')
            ax[idx].set_xticks([])
            ax[idx].set_yticks([])
            idx += 1
        plt.show()

# 4. 데이터 로드 

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
swap_gender_li = [["001498-1", "female"], ["004432", "female"],["005223", "female"], 
                  ['006359', 'male'], ['006360', 'male'], ['006361', 'male'], ['006362', 'male'], ['006363', 'male'], ['006364', 'male'],]
swap_mask_li = ['000020', '004418', '005227']

In [ ]:
seed_everything(config.seed)

In [ ]:
df = pd.read_csv(os.path.join(config.train_data_dir, config.train_data_name))
submission = pd.read_csv(os.path.join(config.submission_data_dir, config.submission_data_name))
submission['idx'] = submission.index

# 5. Main Model

In [ ]:
version = 'main'

In [ ]:
pre_df = preprocessing_df(df = df, swap_gender_li = swap_gender_li)
train_df = make_train_df(df = pre_df, swap_mask_li = swap_mask_li, cfg = config)

all_idx_li = pre_df.index.tolist()
val_idx_li = get_val_idx(df = pre_df, target_col = config.data_split_col)

total_start_time = time.time()

for fold_num in range(1, config.oof + 1):
    fold_start_time = time.time()
    
    # trn, val 데이터 셋
    val_idx = next(val_idx_li)
    trn_idx = list(set(all_idx_li) - set(val_idx.tolist()))
    
    val_id_df = pre_df.iloc[val_idx, :]
    trn_id_df = pre_df.iloc[trn_idx, :]
    
    val_df = train_df.set_index('id').loc[list(set(val_id_df['id'].tolist()) & set(train_df['id'].tolist())), :].reset_index(drop = True)
    trn_df = train_df.set_index('id').loc[list(set(trn_id_df['id'].tolist()) & set(train_df['id'].tolist())), :].reset_index(drop = True)
    
    trn_dataset, train_loader, val_dataset, val_loader = get_trn_val_dataset_dataloader(trn_df = trn_df, 
                                                                                        val_df = val_df, 
                                                                                        cfg = config.mainModel, 
                                                                                        model_name = 'mainModel')
    
    # 모델 정의
    model = CreateModel(cfg = config.mainModel, pretrained = True).to(device)
    criterion = get_criterion(config = config.mainModel)
    optimizer = torch.optim.Adam(model.parameters(), lr=config.mainModel.lr, amsgrad = True)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor = 0.1, eps = 1e-09, patience = 3)
    
    train(model = model,
          optimizer = optimizer, 
          criterion = criterion, 
          train_loader = train_loader, 
          val_loader = val_loader, 
          scheduler = scheduler, 
          config = config.mainModel,
         version = version)
    
    fold_end_time = time.time()
    
    print(f'{fold_num}fold 훈련 시간: {fold_end_time - fold_start_time} \n')
    
    # 메모리 정리
    gc.collect()
    torch.cuda.empty_cache()

total_end_time = time.time()
print(f'총 훈련 시간: {total_end_time - total_start_time}')

In [ ]:
pre_df = preprocessing_df(df = df, swap_gender_li = swap_gender_li)
train_df = make_train_df(df = pre_df, swap_mask_li = swap_mask_li, cfg = config)

all_idx_li = pre_df.index.tolist()
val_idx_li = get_val_idx(df = pre_df, target_col = config.data_split_col)

real_labels = []
pred_labels = []
idx_li = []

for fold_num in range(1, config.oof + 1):
    fold_start_time = time.time()
    
    # val
    val_idx = next(val_idx_li)
    trn_idx = list(set(all_idx_li) - set(val_idx.tolist()))
    
    val_id_df = pre_df.iloc[val_idx, :]
    trn_id_df = pre_df.iloc[trn_idx, :]
    
    val_df = train_df.set_index('id').loc[list(set(val_id_df['id'].tolist()) & set(train_df['id'].tolist())), :].reset_index(drop = True)
    trn_df = train_df.set_index('id').loc[list(set(trn_id_df['id'].tolist()) & set(train_df['id'].tolist())), :].reset_index(drop = True)
    
    trn_dataset, train_loader, val_dataset, val_loader = get_trn_val_dataset_dataloader(trn_df = trn_df, 
                                                                                        val_df = val_df, 
                                                                                        cfg = config.mainModel, 
                                                                                        model_name = 'mainModel')
    model = CreateModel(cfg = config.mainModel, pretrained = False).to(device)
    model.load_state_dict(torch.load(os.path.join(config.mainModel.model_dir, f'{version}_{fold_num}fold_{config.mainModel.model_name}.pt')))
    val_label_pred_li, val_ensemble_pred_li, real_pred_li = get_val_pred_li(model = model, data_loader = val_loader)
    
    real_labels += real_pred_li
    pred_labels += val_label_pred_li
    idx_li += val_df['idx'].tolist()
    
    fold_end_time = time.time()
    
    _acc = get_acc_score(y_true = real_pred_li, y_pred = val_label_pred_li)
    _f1_score = get_f1_score(y_true = real_pred_li, y_pred = val_label_pred_li)
    print(f'{fold_num}fold 훈련 시간: {fold_end_time - fold_start_time}, acc: {_acc}, f1_score: {_f1_score} \n')

In [ ]:
train_f1 = get_f1_score(y_true = real_labels, y_pred = pred_labels)
train_acc = get_acc_score(y_true = real_labels, y_pred = pred_labels)
train_confusion_matrix = pd.DataFrame((confusion_matrix(y_true = real_labels, y_pred = pred_labels)))
print(f'train confusion_matrix')
display(train_confusion_matrix.style.background_gradient(cmap='YlOrRd', axis = 1))
print(f'train fi : {train_f1:.4f}, train acc: {train_acc:.4f} \n')

In [ ]:
get_eval_img_show(train_df = train_df, idx_li = idx_li, pred_labels = pred_labels, config = config.mainModel)

# 6. Male Age Model

In [ ]:
version = 'maleAge'

In [ ]:
pre_df = preprocessing_df(df = df, swap_gender_li = swap_gender_li)
train_df = make_train_df(df = pre_df, swap_mask_li = swap_mask_li, cfg = config)
train_df = train_df[train_df['genders'] == 0].reset_index(drop = True)

all_idx_li = pre_df.index.tolist()
val_idx_li = get_val_idx(df = pre_df, target_col = config.data_split_col)

total_start_time = time.time()

for fold_num in range(1, config.oof + 1):
    fold_start_time = time.time()
    
    # trn, val 데이터 셋
    val_idx = next(val_idx_li)
    trn_idx = list(set(all_idx_li) - set(val_idx.tolist()))
    
    val_id_df = pre_df.iloc[val_idx, :]
    trn_id_df = pre_df.iloc[trn_idx, :]
    
    val_df = train_df.set_index('id').loc[list(set(val_id_df['id'].tolist()) & set(train_df['id'].tolist())), :].reset_index(drop = True)
    trn_df = train_df.set_index('id').loc[list(set(trn_id_df['id'].tolist()) & set(train_df['id'].tolist())), :].reset_index(drop = True)
    
    trn_dataset, train_loader, val_dataset, val_loader = get_trn_val_dataset_dataloader(trn_df = trn_df, 
                                                                                        val_df = val_df, 
                                                                                        cfg = config.maleAgeModel, 
                                                                                        model_name = 'maleAgeModel')
    
    # 모델 정의
    model = CreateModel(cfg = config.maleAgeModel, pretrained = True).to(device)
    criterion = get_criterion(config = config.maleAgeModel)
    optimizer = torch.optim.Adam(model.parameters(), lr=config.maleAgeModel.lr, amsgrad = True)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor = 0.1, eps = 1e-09, patience = 3)
    
    train(model = model,
          optimizer = optimizer, 
          criterion = criterion, 
          train_loader = train_loader, 
          val_loader = val_loader, 
          scheduler = scheduler, 
          config = config.maleAgeModel,
         version = version)
    
    fold_end_time = time.time()
    
    print(f'{fold_num}fold 훈련 시간: {fold_end_time - fold_start_time} \n')
    
    # 메모리 정리
    gc.collect()
    torch.cuda.empty_cache()

total_end_time = time.time()
print(f'총 훈련 시간: {total_end_time - total_start_time}')

In [ ]:
pre_df = preprocessing_df(df = df, swap_gender_li = swap_gender_li)
train_df = make_train_df(df = pre_df, swap_mask_li = swap_mask_li, cfg = config)
train_df = train_df[train_df['genders'] == 0].reset_index(drop = True)

all_idx_li = pre_df.index.tolist()
val_idx_li = get_val_idx(df = pre_df, target_col = config.data_split_col)

real_labels = []
pred_labels = []
idx_li = []

for fold_num in range(1, config.oof + 1):
    fold_start_time = time.time()
    
    # val
    val_idx = next(val_idx_li)
    trn_idx = list(set(all_idx_li) - set(val_idx.tolist()))
    
    val_id_df = pre_df.iloc[val_idx, :]
    trn_id_df = pre_df.iloc[trn_idx, :]
    
    val_df = train_df.set_index('id').loc[list(set(val_id_df['id'].tolist()) & set(train_df['id'].tolist())), :].reset_index(drop = True)
    trn_df = train_df.set_index('id').loc[list(set(trn_id_df['id'].tolist()) & set(train_df['id'].tolist())), :].reset_index(drop = True)
    
    trn_dataset, train_loader, val_dataset, val_loader = get_trn_val_dataset_dataloader(trn_df = trn_df, 
                                                                                        val_df = val_df, 
                                                                                        cfg = config.maleAgeModel, 
                                                                                        model_name = 'maleAgeModel')
    model = CreateModel(cfg = config.maleAgeModel, pretrained = False).to(device)
    model.load_state_dict(torch.load(os.path.join(config.maleAgeModel.model_dir, f'{version}_{fold_num}fold_{config.maleAgeModel.model_name}.pt')))
    val_label_pred_li, val_ensemble_pred_li, real_pred_li = get_val_pred_li(model = model, data_loader = val_loader)
    
    real_labels += real_pred_li
    pred_labels += val_label_pred_li
    idx_li += val_df['idx'].tolist()
    
    fold_end_time = time.time()
    
    _acc = get_acc_score(y_true = real_pred_li, y_pred = val_label_pred_li)
    _f1_score = get_f1_score(y_true = real_pred_li, y_pred = val_label_pred_li)
    print(f'{fold_num}fold 훈련 시간: {fold_end_time - fold_start_time}, acc: {_acc}, f1_score: {_f1_score} \n')

In [ ]:
train_f1 = get_f1_score(y_true = real_labels, y_pred = pred_labels)
train_acc = get_acc_score(y_true = real_labels, y_pred = pred_labels)
train_confusion_matrix = pd.DataFrame((confusion_matrix(y_true = real_labels, y_pred = pred_labels)))
print(f'train confusion_matrix')
display(train_confusion_matrix.style.background_gradient(cmap='YlOrRd', axis = 1))
print(f'train fi : {train_f1:.4f}, train acc: {train_acc:.4f} \n')

In [ ]:
get_eval_img_show(train_df = train_df, idx_li = idx_li, pred_labels = pred_labels, config = config.maleAgeModel)

# 7. Female Age Model

In [ ]:
version = 'femaleAge'

In [ ]:
pre_df = preprocessing_df(df = df, swap_gender_li = swap_gender_li)
train_df = make_train_df(df = pre_df, swap_mask_li = swap_mask_li, cfg = config)
train_df = train_df[train_df['genders'] == 1].reset_index(drop = True)

all_idx_li = pre_df.index.tolist()
val_idx_li = get_val_idx(df = pre_df, target_col = config.data_split_col)

total_start_time = time.time()

for fold_num in range(1, config.oof + 1):
    fold_start_time = time.time()
    
    # trn, val 데이터 셋
    val_idx = next(val_idx_li)
    trn_idx = list(set(all_idx_li) - set(val_idx.tolist()))
    
    val_id_df = pre_df.iloc[val_idx, :]
    trn_id_df = pre_df.iloc[trn_idx, :]
    
    val_df = train_df.set_index('id').loc[list(set(val_id_df['id'].tolist()) & set(train_df['id'].tolist())), :].reset_index(drop = True)
    trn_df = train_df.set_index('id').loc[list(set(trn_id_df['id'].tolist()) & set(train_df['id'].tolist())), :].reset_index(drop = True)
    
    trn_dataset, train_loader, val_dataset, val_loader = get_trn_val_dataset_dataloader(trn_df = trn_df, 
                                                                                        val_df = val_df, 
                                                                                        cfg = config.femaleAgeModel, 
                                                                                        model_name = 'femaleAgeModel')
    
    # 모델 정의
    model = CreateModel(cfg = config.femaleAgeModel, pretrained = True).to(device)
    criterion = get_criterion(config = config.femaleAgeModel)
    optimizer = torch.optim.Adam(model.parameters(), lr=config.femaleAgeModel.lr, amsgrad = True)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor = 0.1, eps = 1e-09, patience = 3)
    
    train(model = model,
          optimizer = optimizer, 
          criterion = criterion, 
          train_loader = train_loader, 
          val_loader = val_loader, 
          scheduler = scheduler, 
          config = config.femaleAgeModel,
         version = version)
    
    fold_end_time = time.time()
    
    print(f'{fold_num}fold 훈련 시간: {fold_end_time - fold_start_time} \n')
    
    # 메모리 정리
    gc.collect()
    torch.cuda.empty_cache()

total_end_time = time.time()
print(f'총 훈련 시간: {total_end_time - total_start_time}')

In [ ]:
pre_df = preprocessing_df(df = df, swap_gender_li = swap_gender_li)
train_df = make_train_df(df = pre_df, swap_mask_li = swap_mask_li, cfg = config)
train_df = train_df[train_df['genders'] == 1].reset_index(drop = True)

all_idx_li = pre_df.index.tolist()
val_idx_li = get_val_idx(df = pre_df, target_col = config.data_split_col)

real_labels = []
pred_labels = []
idx_li = []

for fold_num in range(1, config.oof + 1):
    fold_start_time = time.time()
    
    # val
    val_idx = next(val_idx_li)
    trn_idx = list(set(all_idx_li) - set(val_idx.tolist()))
    
    val_id_df = pre_df.iloc[val_idx, :]
    trn_id_df = pre_df.iloc[trn_idx, :]
    
    val_df = train_df.set_index('id').loc[list(set(val_id_df['id'].tolist()) & set(train_df['id'].tolist())), :].reset_index(drop = True)
    trn_df = train_df.set_index('id').loc[list(set(trn_id_df['id'].tolist()) & set(train_df['id'].tolist())), :].reset_index(drop = True)
    
    trn_dataset, train_loader, val_dataset, val_loader = get_trn_val_dataset_dataloader(trn_df = trn_df, 
                                                                                        val_df = val_df, 
                                                                                        cfg = config.femaleAgeModel, 
                                                                                        model_name = 'femaleAgeModel')
    model = CreateModel(cfg = config.femaleAgeModel, pretrained = False).to(device)
    model.load_state_dict(torch.load(os.path.join(config.femaleAgeModel.model_dir, f'{version}_{fold_num}fold_{config.femaleAgeModel.model_name}.pt')))
    val_label_pred_li, val_ensemble_pred_li, real_pred_li = get_val_pred_li(model = model, data_loader = val_loader)
    
    real_labels += real_pred_li
    pred_labels += val_label_pred_li
    idx_li += val_df['idx'].tolist()
    
    fold_end_time = time.time()
    
    _acc = get_acc_score(y_true = real_pred_li, y_pred = val_label_pred_li)
    _f1_score = get_f1_score(y_true = real_pred_li, y_pred = val_label_pred_li)
    print(f'{fold_num}fold 훈련 시간: {fold_end_time - fold_start_time}, acc: {_acc}, f1_score: {_f1_score} \n')

In [ ]:
train_f1 = get_f1_score(y_true = real_labels, y_pred = pred_labels)
train_acc = get_acc_score(y_true = real_labels, y_pred = pred_labels)
train_confusion_matrix = pd.DataFrame((confusion_matrix(y_true = real_labels, y_pred = pred_labels)))
print(f'train confusion_matrix')
display(train_confusion_matrix.style.background_gradient(cmap='YlOrRd', axis = 1))
print(f'train fi : {train_f1:.4f}, train acc: {train_acc:.4f} \n')

In [ ]:
get_eval_img_show(train_df = train_df, idx_li = idx_li, pred_labels = pred_labels, config = config.femaleAgeModel)

# 8. Ensembel

In [ ]:
def get_eval_data_loader(val_df, cfg, model_name):
    val_dataset = CustomDataset(df = val_df,
                        cfg = cfg,
                        transform = transform[model_name],
                        mode = True,)

    val_loader = DataLoader(val_dataset,
                       batch_size = cfg.batch_size,
                       num_workers = cfg.num_workers,
                       shuffle = False,)

    return val_loader

def get_ans_labels(x):
    if x['pred_labels'] == 0:
        if x['pred_ages'] == 0: return 0
        elif x['pred_ages'] == 1: return 1
        elif x['pred_ages'] == 2: return 2
            
    elif x['pred_labels'] == 1:
        if x['pred_ages'] == 0: return 0
        elif x['pred_ages'] == 1: return 1
        elif x['pred_ages'] == 2: return 2
            
    elif x['pred_labels'] == 2:
        if x['pred_ages'] == 0: return 0
        elif x['pred_ages'] == 1: return 1
        elif x['pred_ages'] == 2: return 2
    
    elif x['pred_labels'] == 3:
        if x['pred_ages'] == 0: return 3
        elif x['pred_ages'] == 1: return 4
        elif x['pred_ages'] == 2: return 5
            
    elif x['pred_labels'] == 4:
        if x['pred_ages'] == 0: return 3
        elif x['pred_ages'] == 1: return 4
        elif x['pred_ages'] == 2: return 5
            
    elif x['pred_labels'] == 5:
        if x['pred_ages'] == 0: return 3
        elif x['pred_ages'] == 1: return 4
        elif x['pred_ages'] == 2: return 5
            
    elif x['pred_labels'] == 6:
        if x['pred_ages'] == 0: return 6
        elif x['pred_ages'] == 1: return 7
        elif x['pred_ages'] == 2: return 8
    
    elif x['pred_labels'] == 7:
        if x['pred_ages'] == 0: return 6
        elif x['pred_ages'] == 1: return 7
        elif x['pred_ages'] == 2: return 8
            
    elif x['pred_labels'] == 8:
        if x['pred_ages'] == 0: return 6
        elif x['pred_ages'] == 1: return 7
        elif x['pred_ages'] == 2: return 8
            
    elif x['pred_labels'] == 9:
        if x['pred_ages'] == 0: return 9
        elif x['pred_ages'] == 1: return 10
        elif x['pred_ages'] == 2: return 11
            
    elif x['pred_labels'] == 10:
        if x['pred_ages'] == 0: return 9
        elif x['pred_ages'] == 1: return 10
        elif x['pred_ages'] == 2: return 11
            
    elif x['pred_labels'] == 11:
        if x['pred_ages'] == 0: return 9
        elif x['pred_ages'] == 1: return 10
        elif x['pred_ages'] == 2: return 11
            
    elif x['pred_labels'] == 12:
        if x['pred_ages'] == 0: return 12
        elif x['pred_ages'] == 1: return 13
        elif x['pred_ages'] == 2: return 14
            
    elif x['pred_labels'] == 13:
        if x['pred_ages'] == 0: return 12
        elif x['pred_ages'] == 1: return 13
        elif x['pred_ages'] == 2: return 14
            
    elif x['pred_labels'] == 14:
        if x['pred_ages'] == 0: return 12
        elif x['pred_ages'] == 1: return 13
        elif x['pred_ages'] == 2: return 14
            
    elif x['pred_labels'] == 15:
        if x['pred_ages'] == 0: return 15
        elif x['pred_ages'] == 1: return 16
        elif x['pred_ages'] == 2: return 17
            
    elif x['pred_labels'] == 16:
        if x['pred_ages'] == 0: return 15
        elif x['pred_ages'] == 1: return 16
        elif x['pred_ages'] == 2: return 17
            
    elif x['pred_labels'] == 17:
        if x['pred_ages'] == 0: return 15
        elif x['pred_ages'] == 1: return 16
        elif x['pred_ages'] == 2: return 17

## 8-1. 평가

In [ ]:
pre_df = preprocessing_df(df = df, swap_gender_li = swap_gender_li)
train_df = make_train_df(df = pre_df, swap_mask_li = swap_mask_li, cfg = config)

all_idx_li = pre_df.index.tolist()
val_idx_li = get_val_idx(df = pre_df, target_col = config.data_split_col)

val_df_li = []

for fold_num in range(1, config.oof + 1):
    fold_start_time = time.time()
    
    # val
    val_idx = next(val_idx_li)
    trn_idx = list(set(all_idx_li) - set(val_idx.tolist()))
    
    val_id_df = pre_df.iloc[val_idx, :]
    trn_id_df = pre_df.iloc[trn_idx, :]
    
    val_df = train_df.set_index('id').loc[list(set(val_id_df['id'].tolist()) & set(train_df['id'].tolist())), :].reset_index(drop = True)
    trn_df = train_df.set_index('id').loc[list(set(trn_id_df['id'].tolist()) & set(train_df['id'].tolist())), :].reset_index(drop = True)
    
    trn_dataset, train_loader, val_dataset, val_loader = get_trn_val_dataset_dataloader(trn_df = trn_df, 
                                                                                        val_df = val_df, 
                                                                                        cfg = config.mainModel, 
                                                                                        model_name = 'mainModel')
    version = 'main'
    model = CreateModel(cfg = config.mainModel, pretrained = False).to(device)
    model.load_state_dict(torch.load(os.path.join(config.mainModel.model_dir, f'{version}_{fold_num}fold_{config.mainModel.model_name}.pt')))
    val_label_pred_li, val_ensemble_pred_li, real_pred_li = get_val_pred_li(model = model, data_loader = val_loader)
    
    val_df['pred_labels'] = val_label_pred_li
    val_df_li.append(val_df.copy())
    
    fold_end_time = time.time()
    
    _acc = get_acc_score(y_true = real_pred_li, y_pred = val_label_pred_li)
    _f1_score = get_f1_score(y_true = real_pred_li, y_pred = val_label_pred_li)
    print(f'{fold_num}fold 훈련 시간: {fold_end_time - fold_start_time}, acc: {_acc}, f1_score: {_f1_score} \n')

male_female_val_df_li = []
for fold_num in range(1, config.oof + 1):
    val_df = val_df_li[fold_num - 1]
    
    # male
    male_val_df = pd.concat([
        val_df[val_df['pred_labels'] == 0],
        val_df[val_df['pred_labels'] == 1],
        val_df[val_df['pred_labels'] == 2],
        
        val_df[val_df['pred_labels'] == 6],
        val_df[val_df['pred_labels'] == 7],
        val_df[val_df['pred_labels'] == 8],
        
        val_df[val_df['pred_labels'] == 12],
        val_df[val_df['pred_labels'] == 13],
        val_df[val_df['pred_labels'] == 14],
    ]).reset_index(drop = True)
    
    male_val_loader = get_eval_data_loader(val_df = male_val_df,
                                           cfg = config.maleAgeModel, 
                                           model_name = 'maleAgeModel')
    
    version = 'maleAge'
    model = CreateModel(cfg = config.maleAgeModel, pretrained = False).to(device)
    model.load_state_dict(torch.load(os.path.join(config.maleAgeModel.model_dir, f'{version}_{fold_num}fold_{config.maleAgeModel.model_name}.pt')))
    val_label_pred_li, val_ensemble_pred_li, real_pred_li = get_val_pred_li(model = model, data_loader = male_val_loader)
    
    male_val_df['pred_ages'] = val_label_pred_li
    male_female_val_df_li.append(male_val_df.copy())
    
    # female
    female_val_df = pd.concat([
        val_df[val_df['pred_labels'] == 3],
        val_df[val_df['pred_labels'] == 4],
        val_df[val_df['pred_labels'] == 5],
        
        val_df[val_df['pred_labels'] == 9],
        val_df[val_df['pred_labels'] == 10],
        val_df[val_df['pred_labels'] == 11],
        
        val_df[val_df['pred_labels'] == 15],
        val_df[val_df['pred_labels'] == 16],
        val_df[val_df['pred_labels'] == 17],
    ]).reset_index(drop = True)
    
    female_val_loader = get_eval_data_loader(val_df = female_val_df,
                                           cfg = config.femaleAgeModel, 
                                           model_name = 'femaleAgeModel')
    
    version = 'femaleAge'
    model = CreateModel(cfg = config.femaleAgeModel, pretrained = False).to(device)
    model.load_state_dict(torch.load(os.path.join(config.femaleAgeModel.model_dir, f'{version}_{fold_num}fold_{config.femaleAgeModel.model_name}.pt')))
    val_label_pred_li, val_ensemble_pred_li, real_pred_li = get_val_pred_li(model = model, data_loader = female_val_loader)
    
    female_val_df['pred_ages'] = val_label_pred_li
    male_female_val_df_li.append(female_val_df.copy())

pred_df = pd.concat(male_female_val_df_li).sort_values('idx').reset_index(drop = True)

In [ ]:
pred_df.head()

In [ ]:
pred_df['ans_labels'] = pred_df.apply(lambda x : get_ans_labels(x), axis = 1)
real_labels = pred_df['labels'].tolist()
pred_labels = pred_df['ans_labels'].tolist()

In [ ]:
train_f1 = get_f1_score(y_true = real_labels, y_pred = pred_labels)
train_acc = get_acc_score(y_true = real_labels, y_pred = pred_labels)
train_confusion_matrix = pd.DataFrame((confusion_matrix(y_true = real_labels, y_pred = pred_labels)))
print(f'train confusion_matrix')
display(train_confusion_matrix.style.background_gradient(cmap='YlOrRd', axis = 1))
print(f'train fi : {train_f1:.4f}, train acc: {train_acc:.4f} \n')

## 8-2. Main Model

In [ ]:
submission_dataset = CustomDataset(df = submission,
                                   cfg = config,
                                   transform = transform['mainModel'],
                                   mode = False,
                                  )

submission_loader = DataLoader(submission_dataset,
                                batch_size = config.mainModel.batch_size,
                                num_workers = config.mainModel.num_workers,
                                shuffle = False,
                             )

main_oof = np.zeros((submission.shape[0], config.mainModel.num_classes))

for fold_num in range(1, config.oof + 1):
    version = 'main'
    model = CreateModel(cfg = config.mainModel, pretrained = False).to(device)
    model.load_state_dict(torch.load(os.path.join(config.mainModel.model_dir, f'{version}_{fold_num}fold_{config.mainModel.model_name}.pt')))
    pred_li, ensemble_pred_li = get_submission_pred_li(model = model, data_loader = submission_loader)
    
    main_oof += ensemble_pred_li / config.oof
    
submission['pred_labels'] = main_oof.argmax(1)

## 8-3. Male Age Model

In [ ]:
male_submission_df = pd.concat([
        submission[submission['pred_labels'] == 0],
        submission[submission['pred_labels'] == 1],
        submission[submission['pred_labels'] == 2],
        
        submission[submission['pred_labels'] == 6],
        submission[submission['pred_labels'] == 7],
        submission[submission['pred_labels'] == 8],
        
        submission[submission['pred_labels'] == 12],
        submission[submission['pred_labels'] == 13],
        submission[submission['pred_labels'] == 14],
    ]).reset_index(drop = True)

submission_dataset = CustomDataset(df = male_submission_df,
                                   cfg = config,
                                   transform = transform['maleAgeModel'],
                                   mode = False,
                                  )

submission_loader = DataLoader(submission_dataset,
                                batch_size = config.maleAgeModel.batch_size,
                                num_workers = config.maleAgeModel.num_workers,
                                shuffle = False,
                             )

male_age_oof = np.zeros((male_submission_df.shape[0], config.maleAgeModel.num_classes))

for fold_num in range(1, config.oof + 1):
    version = 'maleAge'
    model = CreateModel(cfg = config.maleAgeModel, pretrained = False).to(device)
    model.load_state_dict(torch.load(os.path.join(config.maleAgeModel.model_dir, f'{version}_{fold_num}fold_{config.maleAgeModel.model_name}.pt')))
    pred_li, ensemble_pred_li = get_submission_pred_li(model = model, data_loader = submission_loader)
    
    male_age_oof += ensemble_pred_li / config.oof
    
male_submission_df['pred_ages'] = male_age_oof.argmax(1)

## 8-4. Female Age Model

In [ ]:
female_submission_df = pd.concat([
        submission[submission['pred_labels'] == 3],
        submission[submission['pred_labels'] == 4],
        submission[submission['pred_labels'] == 5],
        
        submission[submission['pred_labels'] == 9],
        submission[submission['pred_labels'] == 10],
        submission[submission['pred_labels'] == 11],
        
        submission[submission['pred_labels'] == 15],
        submission[submission['pred_labels'] == 16],
        submission[submission['pred_labels'] == 17],
    ]).reset_index(drop = True)

submission_dataset = CustomDataset(df = female_submission_df,
                                   cfg = config,
                                   transform = transform['femaleAgeModel'],
                                   mode = False,
                                  )

submission_loader = DataLoader(submission_dataset,
                                batch_size = config.femaleAgeModel.batch_size,
                                num_workers = config.femaleAgeModel.num_workers,
                                shuffle = False,
                             )

female_age_oof = np.zeros((female_submission_df.shape[0], config.femaleAgeModel.num_classes))

for fold_num in range(1, config.oof + 1):
    version = 'femaleAge'
    model = CreateModel(cfg = config.femaleAgeModel, pretrained = False).to(device)
    model.load_state_dict(torch.load(os.path.join(config.femaleAgeModel.model_dir, f'{version}_{fold_num}fold_{config.femaleAgeModel.model_name}.pt')))
    pred_li, ensemble_pred_li = get_submission_pred_li(model = model, data_loader = submission_loader)
    
    female_age_oof += ensemble_pred_li / config.oof
    
female_submission_df['pred_ages'] = female_age_oof.argmax(1)

## 8-5. 예측

In [ ]:
submission_df = pd.concat([male_submission_df, female_submission_df]).sort_values('idx').reset_index(drop = True)
submission_df['ans_labels'] = submission_df.apply(lambda x : get_ans_labels(x), axis = 1)
submission_df.head()

In [ ]:
ans_submission_df = submission_df[['ImageID', 'ans_labels']].reset_index(drop = True)
ans_submission_df = ans_submission_df.rename(columns = {'ans_labels' : 'ans'})
ans_submission_df.to_csv(config.file_name, index=False)
ans_submission_df.head()